# Flowline analysis of 4D-STEM data from Nicholas Marchese


Nicholas Marchese - 24th June, 2025

Using the polymers branch  in NM_py4DSTEM

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "2" # Change N to required number of threads, 
                                    # often N=1 is sufficient for most workloads.

# import torch

# torch.set_num_threads(1) # When using .cpu(), PyTorch will also allocate all cores
# #                          # regardless if you set your os environment variable.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import py4DSTEM
import numpy as np
from scipy.ndimage import gaussian_filter
from scipy.ndimage import binary_dilation, binary_erosion
import matplotlib.pyplot as plt
import re
import random
import os
from scipy.signal import medfilt2d

show = py4DSTEM.visualize.show

In [ ]:
import matplotlib.pyplot as plt
plt.close('all')

In [ ]:
py4DSTEM.__version__

In [ ]:
# file_path_data = '/media/cophus/DATA/4DSTEM/TsarfatiYael/dataset19.dm4'
file_data= '/home/yaeltsa/pg3t2/data/07-2024/59_pg3T2_ox_applied0p2mV_insitu_160x160_ss=10nm_CL=2p7m_10umC2_alpha=0p177_bin4_0p013s_spot6__mono20_300kV_15evslit_LN_signal still good/Diffraction SI.dm4'
# file_path_Au = '/home/njmarch/data/PB2T-TEG/Au/STEM SI.dm4'
# file_path_probe = '/media/cophus/DATA/4DSTEM/TsarfatiYael/probe.dm4'
# file_path_output = '/home/njmarch/data/pg3T2/01-09-2025/30/_analysis_'

In [ ]:
# Extract the date after 'data/'
date_match = re.search(r"data/([^/]+)", file_data)
date = date_match.group(1) if date_match else ""
# Extract the name before _ca or _mag after the last slash
name_match = re.search(r"/([^/]+?)_(?:ca|mag)", file_data)
name = name_match.group(1) if name_match else ""
# Concatenate
data_id= f"{date}_{name}"
print(f"data_id is : {data_id}")


match = re.search(r"step_size(\d{1,2})(?!\d)", file_data)
step_size = match.group(1) if match else ""
print(f"step size is : {step_size}")

match = re.search(r"CL(2p[17])", file_data)
cl_raw = match.group(1) if match else ""
cl_value = cl_raw.replace('p', '.') if cl_raw else ""
print(f"CL value is : {cl_value}")

match = re.search(r"[bB]in(?:ning)?[-_]?([248])", file_data)
acq_binning = match.group(1) if match else ""
print(f"Acquisition Binning is : {acq_binning}")

# file_path_output='/Volumes/Yael-2024/07-03-2024/Results/'+'polar_'+data_id+'/'

# if os.path.exists(file_path_output):
#     print("filePath already exist.")
# else:
#     print(file_path_output)
#     os.mkdir(file_path_output) 

In [ ]:
step_size = 10
acq_binning = 4


# User Input

In [ ]:
'''
Real space to q space rotation
old measurement - rotation_calibration_theta= -0.118682 # for 3-15-2021 session TEAMI
'''
rotation_calibration=-6.8 #degrees 
# the rotation between real and reciprocal space in radians
# The TEM or diffraction pattern in EFTEM mode on the K3 is 186.8 degrees clockwise with respect to the scanned images in Microprobe EFSTEM mode
# rotation_calibration_theta= -0.118682 # for 3-15-2021 session TEAMI



In [ ]:
binfactor = 4
pixel_size = 0.005276244931972679 #0.005267464108865549 # in A/pixel
#print((0.005287732135552322-0.005267464108865549)/0.005287732135552322 )

# Import data

In [ ]:

dataset = py4DSTEM.io.import_file(
    file_data,
    data_id = 'datacube_0',
    binfactor = binfactor,
)

In [ ]:
dataset

In [ ]:
dataset

In [ ]:
dataset.data.shape

In [ ]:
dataset.R_pixel_size


In [ ]:
dataset.R_pixel_units


In [ ]:
# dataset.calibration.set_R_pixel_size(6.844)
# dataset.calibration.set_R_pixel_units('nm')
dataset.calibration.set_Q_pixel_size(pixel_size)  # For binfactor of 4
dataset.calibration.set_Q_pixel_units('A^-1')

In [ ]:
# dataset.bin_Q(4)

In [ ]:
dp_max = dataset.get_dp_max();
dp_mean = dataset.get_dp_mean();

In [ ]:
py4DSTEM.show(
    [
        dataset.tree('dp_mean'),
        dataset.tree('dp_max'),
    ]
        )

In [ ]:
mask_hot_pixels =  dataset.tree('dp_mean').data - medfilt2d(dataset.tree('dp_mean').data) > 0.3 * dataset.tree('dp_mean').data
print('Total hot pixels = ' + str(mask_hot_pixels.sum()))

# py4DSTEM.show(
#     mask_hot_pixels,
#     figsize=(6,6),
# )
# Apply mask - this step is not reversible!
dataset.data *= (1 - mask_hot_pixels[None,None,:,:].astype('uint8'))
dataset.get_dp_mean();
dataset.get_dp_max();
py4DSTEM.show(
    dataset.tree('dp_mean'),
    figsize=(6,6),
)

In [ ]:
sliced = dataset.data[::6, ::6]

              
py4DSTEM.show(
    [[
        gaussian_filter(
            py4DSTEM.process.utils.bin2D(sliced[i, j], 2),
            sigma=1,
        )
            for j in range(sliced.shape[1])] for i in range(sliced.shape[0])
    ],
    # vmin=0.3,
    # vmax = 0.994,
    vmin=0.8,
    vmax = 0.98,
    ticks = False,
    axsize=(1, 1),
    cmap='turbo',
)

In [ ]:
# sliced = dataset.data[50:70, 10:20, 40:60, 43:63]

              
# py4DSTEM.show(
#     [[
#         sliced[i, j] for j in range(sliced.shape[1])] for i in range(sliced.shape[0])
#     ],
#     # vmin=0.3,
#     # vmax = 0.994,
#     vmin=0.8,
#     vmax = 0.98,
#     ticks = False,
#     axsize=(2,2),
#     cmap='turbo',
# )

In [ ]:
# probe = py4DSTEM.io.read(
#     file_path_probe,
#     binfactor=2,
# )

# Preprocessing

In [ ]:
# Save the output data and probe
# py4DSTEM.io.save(
#     file_path_output,
#     dataset,
# )

In [ ]:
center = (dataset.shape[2]/2-15, dataset.shape[3]/2-10)
expand_BF = 6
expand_BF_DF = expand_BF+6
geometry_BF = (center, expand_BF)
geometry_DF = (center, (expand_BF_DF, 100))

# Show selected virtual detectors

dataset.position_detector(
    data = dp_max,
    mode = 'circle',
    geometry = geometry_BF,
)

dataset.position_detector(
    data = dp_max,
    mode = 'annulus',
    geometry = geometry_DF,
)

In [ ]:
center_real= [dataset.R_Nx/2-1,dataset.R_Ny/2-10]# right, down
#  Calculate both virtual images
im_bf = dataset.get_virtual_image(
    mode = 'circle',
    geometry = geometry_BF,
    name = 'bright_field'
);
im_adf = dataset.get_virtual_image(
    mode = 'annulus',
    geometry = geometry_DF,
    name = 'dark_field'
);

py4DSTEM.show(
    [im_bf,im_adf],
    figsize=(16,8),
    bordercolor = 'w',
    cmap='viridis',
    title=['Bright Field (BF)','Annular Dark Field (ADF)'],
    axsize = (4,8),
)



In [ ]:
py4DSTEM.visualize.show_points(
    im_bf,
    x=center_real[0],
    y=center_real[0],
    pointcolor='r',
    figsize=(4,4)
)


In [ ]:
# Make mask for dataset which is not over hole region
print(np.min(im_bf.data))
print(np.max(im_bf.data))
print(np.min(im_adf.data))
print(np.max(im_adf.data))
print(dataset.shape)


In [ ]:
xs_reals, ys_reals = np.meshgrid(np.arange(dataset.shape[1]), np.arange(dataset.shape[0])) 

r_max = 107#150#103
mask = (xs_reals - center_real[0])**2 + (ys_reals - center_real[1])**2 > r_max**2
mask = binary_dilation(mask, iterations=3)
print(im_bf.data)
print(np.sum(mask))
print(np.ma.masked_array(im_bf.data, mask))
py4DSTEM.show(
    [np.ma.masked_array(im_bf.data, mask),np.ma.masked_array(im_adf.data, mask)],
    figsize=(16,8),
    bordercolor = 'w',
    cmap='viridis',
    title=['Bright Field (BF)','Annular Dark Field (ADF)'],
    axsize = (4,8),
)

In [ ]:
probe = py4DSTEM.braggvectors.Probe(dp_mean.data)

In [ ]:
probe.measure_disk()

In [ ]:
probe_semiangle, probe_qx0, probe_qy0 = py4DSTEM.process.calibration.get_probe_size(probe.probe)

In [ ]:
probe.get_kernel(
  mode = 'sigmoid',
  origin = (probe_qx0, probe_qy0),
  radii = (
      probe_semiangle*1.0,
      probe_semiangle*3.0,
   )
)

py4DSTEM.visualize.show_kernel(
    probe.kernel,
    R = 30,
    L = 30,
    W = 1,
    figsize = (6,3),
)

In [ ]:
# Choose some diffraction patterns to use for hyperparameter tuning
n_samples_x = 6
n_samples_y = 6
rxs, rys = np.meshgrid(
        np.round(np.linspace(0 + dataset.shape[0] / 10, dataset.shape[0] - dataset.shape[0] / 10, n_samples_x)).astype(int),
        np.round(np.linspace(0 + dataset.shape[1] / 10, dataset.shape[1] - dataset.shape[1] / 10, n_samples_y)).astype(int),
)
rxs = list(rxs.flatten())
rys = list(rys.flatten())
colors = ['r' for i in range(len(rxs))]

py4DSTEM.visualize.show_points(
    im_adf,
    x=rxs,
    y=rys,
    pointcolor=colors,
    figsize=(4,4)
)

py4DSTEM.visualize.show_image_grid(
    get_ar = lambda i:dataset.data[rxs[i],rys[i],:,:],
    H=n_samples_x,
    W=n_samples_y,
    axsize=(3,3),
    power = 0.5,
    get_bordercolor = lambda i:colors[i],
)

In [ ]:
# Find the brightest disk in the image, which will be the unscattered center beam in this data.
(qx0, qy0, _) = py4DSTEM.process.calibration.get_origin(
    dataset,
    r = probe_semiangle,
    rscale = 2.0,
)

In [ ]:
# Fit a plane to the origins
(qx0_fit, qy0_fit, qx0_res, qy0_res) = py4DSTEM.process.calibration.fit_origin(
    (qx0, qy0),
    fitfunction="plane",
)

# plot the original and fitted origin shifts
py4DSTEM.show(
    [
        [qx0 - center[0], qx0_fit - center[0], qx0_res],
        [qy0 - center[1], qy0_fit - center[1], qy0_res],
    ],
    cmap = 'RdBu_r',
    intensity_range='absolute',    
    vmin = -2,
    vmax = 2,
)

In [ ]:
# add the centering calibration to the dataset
dataset.calibration.set_origin((qx0_fit, qy0_fit))

In [ ]:
# First, generate mean diffraction pattern using the centered patterns
dp_mean_centered = dataset.get_virtual_diffraction(
    method = 'mean',
    shift_center = True,
)

In [ ]:
# compare the original, unshifted mean DP to the corrected one:
py4DSTEM.show(
    [
        dataset.tree('dp_mean'),
        dp_mean_centered,
        # [dataset.tree('dp_mean').daata - dp_mean_centered]
    ],
    axsize = (8, 8),
    scaling = 'log',
    vmax = 1,
)

# Here since centered image shifts origin of diffraction pattern for each detector position in realspace, the average value of a pixel over all diffraction patterns will become more gathered according to that shift
# In other words, the diffraction signal will be shifted according to the origin shift, undoing the smearing of signal (here seen as origins being skewed along diagonals)

In [ ]:
# Fitting range for diffraction ring
# q_range = (12,28)  # amorphous halo
q_range = (145 / binfactor, 270 / binfactor)  # inner ring


# This is a test plot, showing you the fitting range
py4DSTEM.process.polar.fit_amorphous_ring(
    dp_mean_centered,
    center = (np.mean(qx0_fit), np.mean(qy0_fit)),
    radial_range = q_range, 
    show_fit_mask = True,
    verbose = False,
    figsize = (8, 8),
);

In [ ]:
params = (np.mean(qx0_fit),
 np.mean(qy0_fit),
 np.float64(40.73975833083233),
 np.float64(39.59277672372462),
 np.float64(-2.729947861471881))

p_dsg = np.array([ 3.76876214e+01,  4.63609962e+00,  3.42475483e+01,  1.57990098e+00,
        1.06194367e+01, -2.10330192e+01,  np.mean(qx0_fit),  np.mean(qy0_fit),
        6.08178012e-04, -2.59723818e-05,  6.32253116e-04])

In [ ]:
# apply the elliptic calibrations
dataset.calibration.set_ellipse(params[2:5])

In [ ]:
dp_mean_ellipse = dataset.get_virtual_diffraction(
    method = 'mean',
    shift_center = True,
)

In [ ]:
# Plot a 2D histogram of the Bragg peaks
py4DSTEM.visualize.show_amorphous_ring_fit(
    dp_mean_ellipse.data,
    fitradii = q_range,
    p_dsg = p_dsg,
    figsize=(8,8),
    ellipse=True,
)

In [ ]:
polardata = py4DSTEM.PolarDatacube(
    dataset,
    qmin = 0,
    qmax = 100.0,
    # qstep = 1,
    qstep = 0.5,
    n_annular = 100,
    two_fold_symmetry = True,
    qscale = 1.0,
)


In [ ]:
# show polar transformation for different probe positions

# x,y = 50, 70
x,y = 30, 30

# x,y = 1,160

# Show pattern in cartesian + polar
py4DSTEM.show( 
    [
        dataset.data[x,y],
        polardata.data[x,y],
    ],
    # intensity_range = 'absolute',
    # vmax = 40,
    cmap = 'turbo',
    axsize = (4,4),
)

# What determines theta = 0 axis?
# Theta is angle from x 

In [ ]:
x, y = 9, 50

rx = np.arange(3)
ry = np.arange(3)
xs = x-2+rx
ys = y-3+ry

In [ ]:
# show polar transformation for different probe positions
# Show pattern in cartesian + polar
for i in xs:
    py4DSTEM.show( 
        [
            [dataset.data[i, j] for j in ys],
            [polardata.data[i, j] for j in ys]
        ],
        # intensity_range = 'absolute',
        # vmax = 40,
        # cmap = 'turbo',
        axsize = (4,4),
        vmin = -0.998,
        vmax = 0.998,
        power=1,
        cmap='viridis',
     
    )

# What determines theta = 0 axis?
# Theta is angle from x 

In [ ]:
# Q Range 0-30

# Reference position (e.g., 120,190 from your comment)
# x, y = 60, 60



positions = [(xv, yv) for xv in xs for yv in ys]

detect_params_1 = {
    'sigma_annular_deg': 3,#4,
    'sigma_radial_px': 1.5, #1.3,
    # 'threshold_abs': 0.855,
    'threshold_abs': 0.1,
    'threshold_prom_radial': 0.01,#0.15
    'threshold_prom_annular': 0.08,
    # 'radial_lower_bound': 0,
    # 'radial_upper_bound': 30,
}

fig, ax = plt.subplots(len(positions), 2, figsize=(10, len(positions)*3))

for i, (x_val, y_val) in enumerate(positions):
    # Cartesian plot (left column)
    cart_data = gaussian_filter(dataset.data[x_val, y_val], sigma=0.5)
    ax[i, 0].imshow(cart_data, cmap='gray', vmin=0, vmax=5)
    ax[i, 0].set_title(f"({x_val},{y_val}) Cartesian")
    ax[i, 0].axis('off')

    # Polar plot with peaks (right column)
    polardata.find_peaks_single_pattern(
        x_val, y_val,
        plot_power_scale=0.5, 
        plot_scale_size=0.05,
        return_background=True,
        figax=(fig, ax[i, 1]),
        **detect_params_1,
        ticks=False,
        vmin = 0,
        vmax = 0.9998,
      
        power=1,
        cmap='viridis',
        plot_result=True,
    )
    ax[i, 1].axvline(x=30, color='red', linestyle='--', linewidth=1)
    ax[i, 1].set_title("Polar")

plt.tight_layout()


In [ ]:
# Q Range 30-60
detect_params_2 = {
    'sigma_annular_deg': 2,#3
    'sigma_radial_px':2,#3
    # 'threshold_abs': 0.855,
    'threshold_abs': 0.01,
    'threshold_prom_radial': 0.1,
    'threshold_prom_annular': 0.1,
    # 'radial_lower_bound': 30,
    # 'radial_upper_bound': 70,
}

# Reference position (e.g., 120,190 from your comment)


positions = [(xv, yv) for xv in xs for yv in ys]


fig, ax = plt.subplots(len(positions), 2, figsize=(10, len(positions)*3))

for i, (x_val, y_val) in enumerate(positions):
    # Cartesian plot (left column)
    cart_data = gaussian_filter(dataset.data[x_val, y_val], sigma=0.1)
    ax[i, 0].imshow(cart_data, cmap='gray', vmin=0, vmax=2)
    ax[i, 0].set_title(f"({x_val},{y_val}) Cartesian")
    ax[i, 0].axis('off')

    # Polar plot with peaks (right column)
    polardata.find_peaks_single_pattern(
        x_val, y_val,
        plot_power_scale=0.5, 
        plot_scale_size=0.05,
        return_background=True,
        figax=(fig, ax[i, 1]),
        **detect_params_2,
        ticks=False,
        vmin = -0.998,
        vmax = 0.998,
        power=1,
        cmap='viridis',
        plot_result=True,
    )
    ax[i, 1].axvline(x=30, color='red', linestyle='--', linewidth=1)
    ax[i, 1].axvline(x=60, color='red', linestyle='--', linewidth=1)

    ax[i, 1].set_title("Polar")

plt.tight_layout()

In [ ]:

positions = [(xv, yv) for xv in xs for yv in ys]

detect_params_3 = {
    'sigma_annular_deg': 2.5,#3.5,
    'sigma_radial_px': 2,
    # 'threshold_abs': 0.855,
    'threshold_abs': 0.1,
    'threshold_prom_radial': 0.25,
    'threshold_prom_annular': 0.2,
    # 'radial_lower_bound': 70,
    # 'radial_upper_bound': 300,
}

fig, ax = plt.subplots(len(positions), 2, figsize=(10, len(positions)*3))

for i, (x_val, y_val) in enumerate(positions):
    # Cartesian plot (left column)
    cart_data = gaussian_filter(dataset.data[x_val, y_val], sigma=0.2)
    ax[i, 0].imshow(cart_data, cmap='gray', vmin=0, vmax=5)
    ax[i, 0].set_title(f"({x_val},{y_val}) Cartesian")
    ax[i, 0].axis('off')

    # Polar plot with peaks (right column)
    polardata.find_peaks_single_pattern(
        x_val, y_val,
        plot_power_scale=0.5, 
        plot_scale_size=0.05,
        return_background=True,
        figax=(fig, ax[i, 1]),
        **detect_params_3,
        ticks=False,
        vmin = 0,
        vmax = 0.9998,
      
        power=1,
        cmap='viridis',
        plot_result=True,
    )
    ax[i, 1].axvline(x=70, color='red', linestyle='--', linewidth=1)
    ax[i, 1].axvline(x=160, color='red', linestyle='--', linewidth=1)

    ax[i, 1].set_title("Polar")

plt.tight_layout()

In [ ]:
kwargs_pass = []

detect_params = {
    'sigma_annular_deg':  detect_params_1 ['sigma_annular_deg'],#4,
    'sigma_radial_px': detect_params_1 ['sigma_radial_px'],# #1.3,
    'threshold_abs': detect_params_1 ['threshold_abs'],# #1.3,
    'threshold_prom_radial': detect_params_1 ['threshold_prom_radial'],# #1.3,
    'threshold_prom_annular': detect_params_1 ['threshold_prom_annular'],# #1.3,
    'radial_lower_bound': 0,# #1.3,
    'radial_upper_bound': 30,
}
kwargs_pass.append(detect_params)

detect_params = {
    'sigma_annular_deg':  detect_params_2 ['sigma_annular_deg'],#4,
    'sigma_radial_px': detect_params_2 ['sigma_radial_px'],# #1.3,
    'threshold_abs': detect_params_2 ['threshold_abs'],# #1.3,
    'threshold_prom_radial': detect_params_2 ['threshold_prom_radial'],# #1.3,
    'threshold_prom_annular': detect_params_2 ['threshold_prom_annular'],# #1.3,
    'radial_lower_bound': 30,
    'radial_upper_bound': 60,
}
kwargs_pass.append(detect_params)

detect_params = {
    'sigma_annular_deg':  detect_params_3 ['sigma_annular_deg'],#4,
    'sigma_radial_px': detect_params_3 ['sigma_radial_px'],# #1.3,
    'threshold_abs': detect_params_3 ['threshold_abs'],# #1.3,
    'threshold_prom_radial': detect_params_3 ['threshold_prom_radial'],# #1.3,
    'threshold_prom_annular': detect_params_3 ['threshold_prom_annular'],# #1.3,
    'radial_lower_bound': 60,
    'radial_upper_bound': 200,
}
kwargs_pass.append(detect_params)

polardata.find_peaks_segmented(
    mask_real=~mask,
    kwargs_pass=kwargs_pass,
)


In [ ]:
# im_polar_sm = polardata.find_peaks_single_pattern(
#     xs[0], ys[3],
#     plot_power_scale = 0.5, 
#     plot_scale_size=0.05,
#     return_background = True,
#     # returnfig = True,
#     # figax = (fig,ax[x_ind, y_ind+1]),
#     **detect_params,
#     ticks = False,
#     vmin = -0.998,
#     vmax = 0.998,
#     # cmap='turbo',
#     # plot_result=True,
#     # plot_smoothed_image = False,
# )
# fig, ax = plt.subplots()
# im = ax.matshow(im_polar_sm)
# plt.colorbar(im)
# plt.show()

In [ ]:
# peaks_polar, sig_bg, sig_bg_mask = polardata.find_peaks_single_pattern(
#     xs[0], ys[3],
#     plot_power_scale = 0.5, 
#     plot_scale_size=0.05,
#     return_background = True,
#     # # returnfig = True,
#     # # figax = (fig,ax[x_ind, y_ind+1]),
#     **detect_params,
#     ticks = False,
#     vmin = -0.998,
#     # vmax = 0.998,
#     # # cmap='turbo',
#     plot_result=True,
#     # plot_smoothed_image = False,
# )

# fig, ax = plt.subplots()
# ax.matshow(im_polar_sm)
# ax.scatter(peaks_polar["qr"], peaks_polar["qt"], c='r')

In [ ]:
# import skimage
# im_peaks = skimage.feature.peak_local_max(
#     im_polar_sm,
#     num_peaks=100,
#     threshold_abs=0.5,
#     exclude_border=False,
# )
# fig, ax = plt.subplots()
# ax.matshow(im_polar_sm)
# ax.scatter(im_peaks[:, 1], im_peaks[:, 0], c='r')

In [ ]:
plt.matshow(~mask)
plt.colorbar()

In [ ]:
polardata.model_radial_background(
    ring_position = [0.01, 0.25, 0.5, 1],
    ring_sigma = [0.1, 0.0055, 0.055, 0.5],
    refine_model = True,
    figsize = (8,3),
)
# (minmum background intensity, background intensity range, sigma, (ring intesity, ring sigma, ring position))

In [ ]:
# This is a 1D histogram of the detected peaks, before refinement
peak_ranges = [
    # polardata.qmin, 0.05,
    0.0225, 0.054,
    0.12, 0.15,  # 1st peak range
    0.22, 0.33,  # 2nd peak range
]
peak_positions_inv_A = [
    0.0085,
    0.0295,
    0.268,  # 2nd peak position
]
polardata.plot_radial_peaks(
    # q_pixel_units = True,
    figsize = (8,2),
    qmin=0,
    v_lines=[
        *peak_ranges,
        # *peak_positions_inv_A

    ]
)

In [ ]:
# This is a 1D histogram of the detected peaks, before refinement
# peak_ranges = [
#     # polardata.qmin, 0.05,
#     0.0283, 0.073,
#     0.113, 0.185, # 1st peak range
#     0.22, 0.32,  # 2nd peak range
# ]
peak_positions_inv_A = [
    0.0085,
    0.0295,
    0.268,  # 2nd peak position
]
polardata.plot_radial_peaks(
    # q_pixel_units = True,
    figsize = (8,2),
    qmin=0,
    v_lines=[
        *peak_ranges,
        # *peak_positions_inv_A

    ]
)
plt.ylim(0, 600)  # If using matplotlib


In [ ]:
# This is a 1D histogram of the detected peaks, before refinement

# peak_positions_inv_A = [
#     0.0295,
#     0.132,  # 1st peak position
#     0.268,  # 2nd peak position
# ]



qmin = 0.06
qmax = 0.4

# Get the radial profile from polardata — adjust based on actual method
# This depends on your library; assuming it's something like:
q_vals, intensities = polardata.plot_radial_peaks(qmin=0.02,qmax=0.4,return_plot_data = True)
# Mask to get intensity values within desired q-range
mask = (q_vals >= qmin) & (q_vals <= qmax)
ymax = 1.1 * np.max(intensities[mask])
plt.xlim(0.02, 0.4)  # If using matplotlib



In [ ]:
polardata.plot_radial_peaks(
    # q_pixel_units = True,
    figsize = (8,2),
    qmin=0,
    v_lines=[
        # *peak_positions_inv_A
        # *peak_positions_inv_A

    ]

)
plt.ylim(0, ymax)  # If using matplotlib
plt.xlim(0.024, 0.4)  # If using matplotlib



In [ ]:
 # This is a 1D histogram of the detected peaks, before refinement

# peak_positions_inv_A = [
#     0.0295,
#     0.132,  # 1st peak position
#     0.268,  # 2nd peak position
# ]

peak_positions_inv_A = [
    (peak_ranges[i] + peak_ranges[i + 1]) / 2
    for i in range(0, len(peak_ranges), 2)
]
lines=[peak_positions_inv_A[0]-0.003,peak_positions_inv_A[1]-0.001,peak_positions_inv_A[2]-0.005]

polardata.plot_radial_peaks(
    # q_pixel_units = True,
    figsize = (8,2),
    qmin=0,    
    v_lines=[
        *lines#*peak_positions_inv_A,
        # *peak_positions_inv_A

    ]

)
plt.ylim(0, 2000)  # If using matplotlib


In [ ]:
peak_positions_A = np.divide(1, lines)
peak_positions_A

In [ ]:
# peak_positions_A = np.divide(1, peak_positions_inv_A)
# peak_positions_A

# Remove Ice 

In [ ]:
peaks_backup = polardata.peaks.copy()

In [ ]:
# Remove ice peaks
# Do by removing peaks above a certain intensity threshold

for i in range(polardata.peaks.shape[0]):
    for j in range(polardata.peaks.shape[1]):
        if len(polardata.peaks[i, j][:]['intensity']) > 0:
            mask_peaks = polardata.peaks[i, j][:]['intensity'] > 2
            print(mask)
            polardata.peaks[i, j].remove(mask_peaks)
        # if len(polardata.peaks[i, j][:]['intensity']) > 0:
        #     print(np.max(polardata.peaks[i, j][:]['intensity']))


# End of Ice Removal

In [ ]:
# polardata.calculate_radial_statistics()

In [ ]:
upsample_factor = 8 

In [ ]:
orient_hist = polardata.make_orientation_histogram(
    [
        [peak_ranges[i], peak_ranges[i+1]] for i in range(0, len(peak_ranges), 2)
    ],
    normalize_intensity_image = True,
    normalize_intensity_stack = False,
    use_refined_peaks = False,
    upsample_factor=upsample_factor,
    orientation_offset_degrees=rotation_calibration

)

In [ ]:
# %%capture orient_hist_summary
# Plot the total signals in each orientation matrix
orient_hist_plot_summary_fig, orient_hist_plot_summary_ax = py4DSTEM.show(
    [x.sum(axis=2) for x in orient_hist],
    cmap='gray',
    # axsize=(3,3),
    ticks = False,
    title=[f'orient_hist {x}' for x in range(len(orient_hist))],
    returnfig=True
)

In [ ]:
# Generate flowline arrays
orient_flowlines = py4DSTEM.process.diffraction.make_flowline_map(
    orient_hist,
    sep_xy = 4.0,
    thresh_seed = 0.1,
    thresh_grow = 0.05,
)

In [ ]:

# orient_flowlines_1 = py4DSTEM.process.diffraction.make_flowline_map(
#     orient_hist[0:1],
#     sep_xy = 4.0,
#     thresh_seed = 0.1,
#     thresh_grow = 0.05,)
# orient_flowlines_2 = py4DSTEM.process.diffraction.make_flowline_map(
#     orient_hist[1:2],
#     sep_xy = 4.0,
#     thresh_seed = 0.1,
#     thresh_grow = 0.05,
#     )
    
orient_flowlines_3 = py4DSTEM.process.diffraction.make_flowline_map(
    orient_hist[2:3],
    sep_xy = 4.0,
    thresh_seed = 0.1,
    thresh_grow = 0.05,
    )

In [ ]:
orient_concat = np.concatenate(
    [
        orient_flowlines_1,
        orient_flowlines_2,
        orient_flowlines_3,
    ],
    axis=0
)

In [ ]:
int_range = [0,0.1]
figsize = (6,6)

im_flowlines = []
titles = []
for i in range(len(orient_flowlines)):
    im = py4DSTEM.process.diffraction.make_flowline_rainbow_image(
    orient_flowlines[i][None,:,:,:],
    int_range=int_range,
    greyscale=False,
    white_background=False,
    plot_images=False,
    sum_radial_bins=True,
    figsize = figsize,
    )
    im_flowlines.append(im[0])
    titles.append(f"flowline {i}")

py4DSTEM.show(
    im_flowlines,
    titles
)

In [ ]:

i = 0  # choose which flowline to display
im = im_flowlines[i]

# If grayscale or single-channel: use vmin/vmax to adjust brightness/contrast
plt.figure(figsize=(6, 6))
plt.imshow(im, cmap='viridis', vmin=0.0, vmax=0.0001)  # adjust vmin/vmax as needed
# plt.title(f"Flowline {i}")
plt.axis('off')
plt.gca().set_aspect('equal')  # Enforce 1:1 aspect ratio
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

i = 1  # index of the flowline image to display
im = im_flowlines[i]

fig = plt.figure(figsize=(6, 6))  # Set exact figure size
ax = fig.add_axes([0, 0, 1, 1])   # Full figure area (left, bottom, width, height)

ax.imshow(im, cmap='viridis', vmin=-1, vmax=0.0000001)  # Adjust contrast as needed
ax.set_aspect('equal')          # 1:1 aspect ratio
ax.axis('off')                  # Remove axes and ticks

plt.show()


In [ ]:
i = 2  # index of the flowline image to display
im = im_flowlines[i]

fig = plt.figure(figsize=(6, 6))  # Set exact figure size
ax = fig.add_axes([0, 0, 1, 1])   # Full figure area (left, bottom, width, height)

ax.imshow(im, cmap='viridis', vmin=-0.1, vmax=0.001,)  # Adjust contrast as needed
ax.set_aspect('equal')          # 1:1 aspect ratio
ax.axis('off')                  # Remove axes and ticks

plt.show()



In [ ]:

im = py4DSTEM.process.diffraction.make_flowline_rainbow_image(
    orient_flowlines_3,
    int_range=int_range,
    greyscale=False,
    white_background=False,
    plot_images=True,
    sum_radial_bins=True,
    figsize = figsize,
)



In [ ]:
fig = plt.figure(figsize=(6, 6))  # Set exact figure size
ax = fig.add_axes([0, 0, 1, 1])   # Full figure area (left, bottom, width, height)

ax.imshow(im[0], cmap='viridis', vmin=-0.1, vmax=0.001,)  # Adjust contrast as needed
ax.set_aspect('equal')          # 1:1 aspect ratio
ax.axis('off')                  # Remove axes and ticks

plt.show()

# Finer Orientation Mask 

In [ ]:
# Generate flowline arrays
orient_flowlines_fine = py4DSTEM.process.diffraction.make_flowline_map(
    orient_hist,
    sep_xy = 4.0,
    thresh_seed = 0.01,
    thresh_grow = 0.005,
)

In [ ]:

orient_flowlines_1_fine = py4DSTEM.process.diffraction.make_flowline_map(
    orient_hist[0:1],
    sep_xy = 4.0,
    thresh_seed = 0.01,
    thresh_grow = 0.005,)
orient_flowlines_2_fine = py4DSTEM.process.diffraction.make_flowline_map(
    orient_hist[1:2],
    sep_xy = 4.0,
    thresh_seed = 0.01,
    thresh_grow = 0.005,)
    
orient_flowlines_3_fine = py4DSTEM.process.diffraction.make_flowline_map(
    orient_hist[2:3],
    sep_xy = 4.0,
    thresh_seed = 0.01,
    thresh_grow = 0.005,
    )

In [ ]:
orient_concat = np.concatenate(
    [
        orient_flowlines_1_fine,
        orient_flowlines_2_fine,
        orient_flowlines_3_fine,
    ],
    axis=0
)

In [ ]:
int_range = [0,0.1]
figsize = (6,6)

im_flowlines_fine = []
titles = []
for i in range(len(orient_flowlines_fine)):
    im = py4DSTEM.process.diffraction.make_flowline_rainbow_image(
    orient_flowlines_fine[i][None,:,:,:],
    int_range=int_range,
    greyscale=False,
    white_background=False,
    plot_images=False,
    sum_radial_bins=True,
    figsize = figsize,
    )
    im_flowlines_fine.append(im[0])
    titles.append(f"flowline {i}")

py4DSTEM.show(
    im_flowlines,
    titles
)

In [ ]:

i = 0  # choose which flowline to display
im = im_flowlines_fine[i]

# If grayscale or single-channel: use vmin/vmax to adjust brightness/contrast
plt.figure(figsize=(6, 6))
plt.imshow(im, cmap='viridis', vmin=0.0, vmax=0.05)  # adjust vmin/vmax as needed
# plt.title(f"Flowline {i}")
plt.axis('off')
plt.gca().set_aspect('equal')  # Enforce 1:1 aspect ratio
plt.tight_layout()
plt.show()


In [ ]:

i = 1  # choose which flowline to display
im = im_flowlines_fine[i]

# If grayscale or single-channel: use vmin/vmax to adjust brightness/contrast
plt.figure(figsize=(6, 6))
plt.imshow(im, cmap='viridis', vmin=0.0, vmax=0.01)  # adjust vmin/vmax as needed
# plt.title(f"Flowline {i}")
plt.axis('off')
plt.gca().set_aspect('equal')  # Enforce 1:1 aspect ratio
plt.tight_layout()
plt.show()


In [ ]:

i = 2  # choose which flowline to display
im = im_flowlines_fine[i]

# If grayscale or single-channel: use vmin/vmax to adjust brightness/contrast
plt.figure(figsize=(6, 6))
plt.imshow(im, cmap='viridis', vmin=0.0, vmax=0.0001)  # adjust vmin/vmax as needed
# plt.title(f"Flowline {i}")
plt.axis('off')
plt.gca().set_aspect('equal')  # Enforce 1:1 aspect ratio
plt.tight_layout()
plt.show()


# End of Finer Orientatio mask '


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

mask_centers =[
    (250,900),
    (350,930),
    (430,950),
    (600,950),
    ] 
mask_radius=10
i = 0  # index of the flowline image to display
im = im_flowlines_fine[i]

fig = plt.figure(figsize=(6, 6))  # Set exact figure size
ax = fig.add_axes([0, 0, 1, 1])   # Full figure area (left, bottom, width, height)

ax.imshow(im, cmap='viridis', vmin=0.0, vmax=0.001)  # Adjust contrast as needed
ax.set_aspect('equal')          # 1:1 aspect ratio
ax.axis('off')                  # Remove axes and ticks
# ax.add_patch(circle)
for (x, y) in mask_centers:
    circle = Circle(
        (x, y),
        radius=mask_radius,
        facecolor='white',
        edgecolor='red',
        linewidth=1.5,
        alpha=0.2
    )
    ax.add_patch(circle)
plt.show()


In [ ]:
upsample_factor

# point 0

In [ ]:
n=0 # first point 
mask_center = (mask_centers[n][1]/upsample_factor,mask_centers[n][0]/upsample_factor)
# print(mask_center)
mask_radius = 2

In [ ]:
img = dataset.tree('dark_field')
# print("Shape of image:", img.shape)

In [ ]:
py4DSTEM.visualize.show(
    dataset.tree('dark_field'),
    circle={
        'center': mask_center,  # (row, col) order = (y, x)
        'R': mask_radius,
        'alpha': 0.3,
        'fill': True
    },
    ticks=False,
)



In [ ]:
ryy,rxx = np.meshgrid(
    np.arange(dataset.R_Ny),
    np.arange(dataset.R_Nx),
)
rrr = np.hypot( rxx-mask_center[0] , ryy-mask_center[1] )
mask = rrr < mask_radius

# show

show(
    dataset.tree('dark_field'),
    mask = mask,
    vmin=0,
    vmax=0.1,
    scaling='log'
)

In [ ]:
selected_area_diffraction_01 = dataset.get_virtual_diffraction(
    method = 'mean',
    mask = mask,
    name = 'selected_area_diffraction_01'
)

# show
py4DSTEM.visualize.show(
    selected_area_diffraction_01,vmax=0.9975,ticks=False, 
)


# point 1

In [ ]:
n=1 # first point 
mask_center = (mask_centers[n][1]/upsample_factor,mask_centers[n][0]/upsample_factor)
# print(mask_center)
mask_radius = 2
img = dataset.tree('dark_field')
# print("Shape of image:", img.shape)
py4DSTEM.visualize.show(
    dataset.tree('dark_field'),
    circle={
        'center': mask_center,  # (row, col) order = (y, x)
        'R': mask_radius,
        'alpha': 0.3,
        'fill': True
    },
    ticks=False,
)


ryy,rxx = np.meshgrid(
    np.arange(dataset.R_Ny),
    np.arange(dataset.R_Nx),
)
rrr = np.hypot( rxx-mask_center[0] , ryy-mask_center[1] )
mask = rrr < mask_radius

# show

show(
    dataset.tree('dark_field'),
    mask = mask,
    vmin=0,
    vmax=0.1,
    scaling='log'
);
selected_area_diffraction_01 = dataset.get_virtual_diffraction(
    method = 'mean',
    mask = mask,
    name = 'selected_area_diffraction_01'
);

# show
py4DSTEM.visualize.show(
    selected_area_diffraction_01,vmax=0.9975,ticks=False, 
)


# point 2

In [ ]:
n=2 # first point 
mask_center = (mask_centers[n][1]/upsample_factor,mask_centers[n][0]/upsample_factor)
# print(mask_center)
mask_radius = 2
img = dataset.tree('dark_field')
# print("Shape of image:", img.shape)
py4DSTEM.visualize.show(
    dataset.tree('dark_field'),
    circle={
        'center': mask_center,  # (row, col) order = (y, x)
        'R': mask_radius,
        'alpha': 0.3,
        'fill': True
    },
    ticks=False,
)


ryy,rxx = np.meshgrid(
    np.arange(dataset.R_Ny),
    np.arange(dataset.R_Nx),
)
rrr = np.hypot( rxx-mask_center[0] , ryy-mask_center[1] )
mask = rrr < mask_radius

# show

show(
    dataset.tree('dark_field'),
    mask = mask,
    vmin=0,
    vmax=0.1,
    scaling='log'
)
selected_area_diffraction_01 = dataset.get_virtual_diffraction(
    method = 'mean',
    mask = mask,
    name = 'selected_area_diffraction_01'
)

# show
py4DSTEM.visualize.show(
    selected_area_diffraction_01,vmax=0.9975,ticks=False, 
)


# correlation 

In [ ]:
# orient_corr_50 = py4DSTEM.process.diffraction.orientation_correlation(
#     orient_hist_upsample_1,
#     radius_max=50, 
# )

In [ ]:
#yael
upsample_factor_corr=8
step_size=15
rotation_calibration= -6.8



orient_hist_corr = polardata.make_orientation_histogram(
    [
        [peak_ranges[i], peak_ranges[i+1]] for i in range(0, len(peak_ranges), 2)
    ],
    normalize_intensity_image = True,
    normalize_intensity_stack = False,
    use_refined_peaks = False,
    theta_step_deg=1,
    upsample_factor=upsample_factor_corr,
    orientation_offset_degrees=rotation_calibration
)


In [ ]:
orient_corr = py4DSTEM.process.diffraction.orientation_correlation(
    orient_hist_corr,
    # radius_max=50,
)



In [ ]:

fig,ax = py4DSTEM.process.diffraction.plot_orientation_correlation(
    orient_corr,
    calculate_coefs=True,
    prob_range = [0.1,10.0],
    pixel_size = int(step_size) /upsample_factor_corr,
    pixel_units='nm',
    returnfig=True,fontsize=18,
)